In [8]:
import os
import numpy as np
from collections import deque

In [3]:
# take a nii file
data_path = "D:\dev\CT-Covid-19-August2020"
example_filename = os.path.join(data_path, 'volume-covid19-A-0000.nii.gz')

# load image
import nibabel as nib
img = nib.load(example_filename)

# show shapes
img.shape

(512, 512, 341)

In [13]:
# growing function
class RegionGrow3D():
    """
    documentation
    """

    def __init__(self, images, masks, upperThreshold, lowerThreshold, neighborMode):
        self.images = images
        self.masks = masks
        self.outputMask = np.zeros_like(self.slices)
        self.upperThreshold = upperThreshold
        self.lowerThreshold = lowerThreshold
        self.neighborMode = neighborMode
        self.queue = deque()

    def main(self, seed):

        newItem = np.zeros(3)
        neighbors = np.zeros(26,3)
        newItem = seed

        self.outputMask[newItem[0], newItem[1], newItem[2]] = 1
        self.queue.append((seed[0], seed[1], seed[2]))

        while len(self.queue) != 0:
            newItem = self.queue.pop()
            if self.neighborMode == "26n":
                neighbors = [[newItem[0]-1, newItem[1]-1, newItem[2]-1],   [newItem[0]-1, newItem[1]-1, newItem[2]],   [newItem[0]-1, newItem[1]-1, newItem[2]+1],
                             [newItem[0]-1, newItem[1], newItem[2]-1],     [newItem[0]-1, newItem[1], newItem[2]],     [newItem[0]-1, newItem[1], newItem[2]+1],
                             [newItem[0]-1, newItem[1]+1, newItem[2]-1],   [newItem[0]-1, newItem[1]+1, newItem[2]],   [newItem[0]-1, newItem[1]+1, newItem[2]+1],
                             [newItem[0], newItem[1]-1, newItem[2]-1],     [newItem[0], newItem[1]-1, newItem[2]],     [newItem[0], newItem[1]-1, newItem[2]+1],
                             [newItem[0], newItem[1], newItem[2]-1],       [newItem[0], newItem[1], newItem[2]+1],     [newItem[0], newItem[1]+1, newItem[2]-1],
                             [newItem[0], newItem[1]+1, newItem[2]],       [newItem[0], newItem[1]+1, newItem[2]+1],   [newItem[0]+1, newItem[1]-1, newItem[2]-1],
                             [newItem[0]+1, newItem[1]-1, newItem[2]],     [newItem[0]+1, newItem[1]-1, newItem[2]+1], [newItem[0]+1, newItem[1], newItem[2]-1],
                             [newItem[0]+1, newItem[1], newItem[2]],       [newItem[0]+1, newItem[1], newItem[2]+1],   [newItem[0]+1, newItem[1]+1, newItem[2]-1],
                             [newItem[0]+1, newItem[1]+1, newItem[2]],     [newItem[0]+1, newItem[1]+1, newItem[2]+1]]
                for neighbor in neighbors:
                    self.checkNeighbour(neighbor[0], neighbor[1], neighbor[2])
            elif self.neighborMode == "6n":
                self.checkNeighbour(newItem[0], newItem[1], newItem[2]-1)
                self.checkNeighbour(newItem[0], newItem[1], newItem[2]+1)
                self.checkNeighbour(newItem[0], newItem[1]-1, newItem[2])
                self.checkNeighbour(newItem[0], newItem[1]+1, newItem[2])
                self.checkNeighbour(newItem[0]-1, newItem[1], newItem[2])
                self.checkNeighbour(newItem[0]+1, newItem[1], newItem[2])
        return self.outputMask

    # todo check this 513 if it is meaningful
    def checkNeighbour(self, z, y, x):
        if (x < 513 and y < 513 and z < 315
            and x > -1 and y > -1 and z > -1
            and self.masks[z,y,x] == 1):
            intensity = self.images[z, y, x]
            if self.isIntensityAcceptable(intensity) and self.outputMask[z,y,x] == 0:
                self.outputMask[z,y,x] = 1
                self.queue.append((z, y, x))

    def isIntensityAcceptable(self, intensity):
        if ((intensity < self.upperThreshold) and (intensity > self.lowerThreshold)):
            return True
        else:
          return False

In [ ]:
# todo insert seed and edges